In [ ]:
!pip install rembg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
import torch
import os
import io
from PIL import Image, ImageEnhance, ImageFilter, ImageDraw
from torchvision import datasets
from rembg import remove
from matplotlib import pyplot as plt
from IPython.display import display

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
FOLDERPATH = 'Colab\ Notebooks/Poster'

In [ ]:
%cd drive/MyDrive/$FOLDERPATH

/content/drive/.shortcut-targets-by-id/1OC3Immm4L7H1Rp4Po9pQSFc3-pDwQpi4/Poster


In [ ]:
%ls

 boxblur/                                model_Densenet121_weights/
 box_blur_30/                            model_Densenet121_weights.pth
 change_style.ipynb                      model_EfficientNetb0_weights.pth
 checkpoint.pt                           model_EfficientNetb4_weights/
 contour/                                model_EfficientNetb4_weights.pth
'Copy of ensemble_0921.ipynb'            module_ResNet50.ipynb
'Copy of module_ResNet50.ipynb'          module_VIiT_0908.ipynb
 correct_mile/                           new_test/
 correct_mile_GEN                        remove_back/
 Dataset/                                remove_back_has_back/
 Densenet121.ipynb                       resnet10_max+avgpool_1e4_100_2classes_0918.pth
 Densenet121_weights_920/                resnet18_avgpool_1e3_100.pth
 Densenet121_weights_920.pth             resnet18_maxpool_1e3_100_2classes.pth
 Densenet121_weights_earlystop_920.pth   resnet50_2classes_weights_0918/
 detail_sd_smooth_por/               

In [ ]:
# Check if gpu is available
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(f'計算裝置:{device}')

計算裝置:cuda


In [ ]:
# need number
def contrast_ef(image, factor):
	enhancer = ImageEnhance.Contrast(image)
	return enhancer.enhance(factor)

def bright_ef(image, factor):
	enhancer = ImageEnhance.Brightness(image)
	return enhancer.enhance(factor)

def color_ef(image, factor):
	enhancer = ImageEnhance.Color(image)
	return enhancer.enhance(factor)

def sharp_ef(image, factor):
	enhancer = ImageEnhance.Sharpness(image)
	return enhancer.enhance(factor)

def gaussian_blur_ef(image, radius):
	return image.filter(ImageFilter.GaussianBlur(radius))

# no number
def smooth_ef(image):
	return image.filter(ImageFilter.SMOOTH)

def detail_ef(image):
	return image.filter(ImageFilter.DETAIL)

def sharp_ef_2(image):
	return image.filter(ImageFilter.SHARPEN)

def blur_ef(image):
	return image.filter(ImageFilter.BLUR)

def detail_ef(image):
	return image.filter(ImageFilter.DETAIL)

def smoother_ef(image):
    return image.filter(ImageFilter.SMOOTH_MORE)

# shadow/ light
def highlights_shadows_sd(image, highlight_factor=1.1, shadow_factor=0.9):
	img_array = np.array(image)
	# RGB
	brightness = np.mean(img_array, axis=2)
	highlight_threshold = np.percentile(brightness, 99)
	shadow_threshold = np.percentile(brightness, 1)
	# light
	img_array[brightness >= highlight_threshold] = np.clip(
		img_array[brightness >= highlight_threshold] * highlight_factor, 0, 255
	)
	# shadow
	img_array[brightness <= shadow_threshold] = np.clip(
		img_array[brightness <= shadow_threshold] * shadow_factor, 0, 255
	)
	return Image.fromarray(img_array.astype(np.uint8))

def highlights_shadows_por(image, highlight_factor=0.9, shadow_factor=1.1):
	img_array = np.array(image)
	# RGB
	brightness = np.mean(img_array, axis=2)
	highlight_threshold = np.percentile(brightness, 99)
	shadow_threshold = np.percentile(brightness, 1)
	# light
	img_array[brightness >= highlight_threshold] = np.clip(
		img_array[brightness >= highlight_threshold] * highlight_factor, 0, 255
	)
	# shadow
	img_array[brightness <= shadow_threshold] = np.clip(
		img_array[brightness <= shadow_threshold] * shadow_factor, 0, 255
	)
	return Image.fromarray(img_array.astype(np.uint8))

# up/ mirror
def mirror_up(image):
	return image.transpose(Image.FLIP_TOP_BOTTOM)

def rotate_225(image):
	return image.rotate(225)

# gray
def gray_ef(image):
	return image.convert('L')

# filter(線稿 馬賽克 加強邊緣)
def filter_ef(image):
	return image.filter(ImageFilter.CONTOUR)

def filter_ef_2(image):
	return image.filter(ImageFilter.BoxBlur(radius=10))

def filter_ef_3(image):
	return image.filter(ImageFilter.EDGE_ENHANCE_MORE)

def filter_ef_4(image):
	return image.filter(ImageFilter.EMBOSS)

def filter_ef_22(image):
	return image.filter(ImageFilter.BoxBlur(radius=30))

# hide
def add_occlusion(image, box):
	draw = ImageDraw.Draw(image)
	box = (420, 240, 460, 260)
	draw.rectangle(box, fill=(0, 0, 0))

In [ ]:
# remove_back
def remove_back(image):
    img_bytes = io.BytesIO()
    image.save(img_bytes, format='PNG')
    img_bytes_arr = img_bytes.getvalue()
    image_no_back_bytes = remove(img_bytes_arr)
    image_no_back = Image.open(io.BytesIO(image_no_back_bytes))
    return image_no_back

	# image_no_back_bytes = remove(image)
	# image_no_back = Image.open(io.BytesIO(image_no_back_bytes))
	# return image_no_back


In [ ]:
# PATH_IN = 'correct_mile/Portrait'
# PATH_OUT = 'remove_back/Portrait'
PATH_IN = os.path.join("correct_mile", "Stable Diffusion")
PATH_OUT = os.path.join("remove_back", "Stable Diffusion")

# 創建输出文件夹（如果不存在）
os.makedirs(PATH_OUT, exist_ok=True)

In [ ]:
# # PATH_IN = 'remove_back/Portrait'
# # PATH_OUT = 'remove_back_has_back/Portrait'
# PATH_IN = os.path.join("remove_back", "Stable Diffusion")
# PATH_OUT = os.path.join("remove_back_has_back", "Stable Diffusion")

# os.makedirs(PATH_OUT, exist_ok=True)

In [ ]:
# 檢查路徑

In [ ]:
for filename in os.listdir(PATH_IN):
    open_file = os.path.join(PATH_IN, filename)
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image = Image.open(open_file)
        # SD
        # image_after = contrast_ef(image, 1.3)
        # image_after = bright_ef(image_after, 1.03)
        # image_after = highlights_shadows(image_after)
        # image_after = color_ef(image_after, 1.1)
        # image_after = sharp_ef(image_after, 3)
        # image_after = sharp_ef_2(image_after)
        # image_after = detail_ef(image_after)

        # por
        # image_after = contrast_ef(image, 0.8)
        # image_after = bright_ef(image_after, 1.03)
        # image_after = highlights_shadows(image_after)
        # image_after = color_ef(image_after, 0.8)
        # image_after = smooth_ef(image_after)
        # image_after = gaussian_blur_ef(image_after, 1)
        # image_after = smoother_ef(image_after)

        # image_after = remove_back(image)

        # save_file = os.path.join(PATH_OUT, filename)
        # image_after.save(save_file)
        # print(f"已保存文件: {save_file}")

        # save_file = save_file if save_file.endswith('.jpg') else save_file + '.jpg'
        # image.save(save_file, quality=45)

In [ ]:
def add_white_background(image):
    img = image.convert("RGBA")
    white_background = Image.new("RGBA", img.size, (255, 255, 255, 255))
    # 將圖像黏貼到白色背景上，保持透明部分
    white_background.paste(img, (0, 0), img)
    # 轉換為 RGB 模式，去除透明度
    return white_background.convert("RGB")

In [ ]:
# for filename in os.listdir(PATH_IN):
#     if filename.endswith('.png'):
#         open_file = os.path.join(PATH_IN, filename)
#         image = Image.open(open_file)
#         image_back = add_white_background(image)


#         save_file = os.path.join(PATH_OUT, filename.replace('.png', '.jpg'))
#         image_back.save(save_file, format='JPEG')
#         print(f"已保存文件: {save_file}")

In [ ]:
# 有跳過文件時使用
# for filename in os.listdir(PATH_IN):
#     # 構建文件的完整路徑
#     open_file = os.path.join(PATH_IN, filename)

#     # 檢查文件是否為圖片（處理 .jpg 或 .png 文件）
#     if filename.endswith('.jpg') or filename.endswith('.png'):
#         save_file = os.path.join(PATH_OUT, filename)

#         # 如果目標文件夾中已存在該文件，則跳過
#         if not os.path.exists(save_file):
#             # 打開圖片
#             image = Image.open(open_file)
#             # SD
#             # por


#             # 保存圖片到輸出路徑
#             image.save(save_file)
#             print(f"已保存文件: {save_file}")
#         else:
#             print(f"文件已存在，跳過: {save_file}")

In [ ]:
# 列出文件夾中的所有文件
all_files = os.listdir(PATH_OUT)

# 過濾出圖片文件 (只統計 .jpg 和 .png 的文|件)
image_files = [f for f in all_files if f.endswith(('.jpg', '.png'))]

# 打印總圖片數
print(f"文件夹中共有 {len(image_files)} 张图片")

文件夹中共有 177 张图片


In [ ]:
# 列出文件夾中的所有文件
all_files = os.listdir(PATH_OUT)

# 過濾出圖片文件 (只統計 .jpg 和 .png 的文件)
image_files = [f for f in all_files if f.endswith(('.webp'))]

# 打印總圖片數
print(f"文件夾中共有 {len(image_files)} 張圖片")

文件夹中共有 0 张图片


### **Final change**

In [ ]:
PATH_IN = 'final_3/Portrait'
PATH_OUT = 'final_4/Portrait'

# 創建輸出文件夾（如果不存在）
os.makedirs(PATH_OUT, exist_ok=True)

In [ ]:
# remove_back
def remove_back(image):
    img_bytes = io.BytesIO()
    image.save(img_bytes, format='PNG')
    img_bytes_arr = img_bytes.getvalue()
    image_no_back_bytes = remove(img_bytes_arr)
    image_no_back = Image.open(io.BytesIO(image_no_back_bytes))
    return image_no_back

In [ ]:
for filename in os.listdir(PATH_IN):
    open_file = os.path.join(PATH_IN, filename)
    if filename.endswith('.JPG') or filename.endswith('.webp') or filename.endswith('.jpg'):
        image = Image.open(open_file)

        # por
        image_after = contrast_ef(image, 0.8)
        image_after = bright_ef(image_after, 1.03)
        image_after = highlights_shadows_por(image_after)
        image_after = color_ef(image_after, 0.8)
        image_after = smooth_ef(image_after)
        image_after = gaussian_blur_ef(image_after, 1)

        image_after = smoother_ef(image_after)

        # image_after = filter_ef_2(image_after)

        # image_after = remove_back(image)

        save_file = os.path.join(PATH_OUT, filename)

        # save_file = save_file if save_file.endswith('.png') else save_file.replace('.JPG', '.png').replace('.webp', '.png').replace('.jpg', '.png')
        # image_after.save(save_file, format='PNG')
        # print(f"已保存文件: {save_file}")

        save_file = os.path.join(PATH_OUT, filename)
        save_file = save_file if save_file.endswith('.jpg') else save_file.replace('.JPG', '.jpg').replace('.webp', '.jpg').replace('.png', '.jpg')
        image_after.save(save_file, format='JPEG', quality=45)
        print(f"已保存文件: {save_file}")


已保存文件: final_4/Portrait/006_4000 × 6000.jpg
已保存文件: final_4/Portrait/008_4000 × 6000.jpg
已保存文件: final_4/Portrait/005_4000 × 6000.jpg
已保存文件: final_4/Portrait/011_4000 × 6000.jpg
已保存文件: final_4/Portrait/004_4000 × 6000.jpg
已保存文件: final_4/Portrait/013_4000 × 6000.jpg
已保存文件: final_4/Portrait/014_4000 × 6000.jpg
已保存文件: final_4/Portrait/009_4000 × 6000.jpg
已保存文件: final_4/Portrait/010_4000 × 6000.jpg
已保存文件: final_4/Portrait/020_2415 × 3680.jpg
已保存文件: final_4/Portrait/003_2624 × 3936.jpg
已保存文件: final_4/Portrait/017_4016 × 6016.jpg
已保存文件: final_4/Portrait/018_4016 × 6016.jpg
已保存文件: final_4/Portrait/019_4016 × 6016.jpg
已保存文件: final_4/Portrait/001_1433 × 1305.jpg
已保存文件: final_4/Portrait/002_2594 × 3175.jpg
已保存文件: final_4/Portrait/007_3549 × 4504.jpg
已保存文件: final_4/Portrait/012_3326 × 3028.jpg
已保存文件: final_4/Portrait/015_3295 × 3253.jpg
已保存文件: final_4/Portrait/016_2771 × 3687.jpg
已保存文件: final_4/Portrait/1.jpg
已保存文件: final_4/Portrait/2.jpg
已保存文件: final_4/Portrait/3.jpg
已保存文件: final_4/Portrait/4.jpg


In [ ]:
# 列出文件夾中的所有文件
all_files = os.listdir(PATH_OUT)

# 過濾出圖片文件 (只統計 .jpg 和 .png 的文|件)
image_files = [f for f in all_files if f.endswith(('.jpg', '.png'))]

# 打印總圖片數
print(f"文件夹中共有 {len(image_files)} 张图片")

文件夹中共有 200 张图片


In [ ]:
PATH_IN = 'final_pre_4/Portrait'
PATH_OUT = 'final_4/Portrait'

# 創建输出文件夹（如果不存在）
os.makedirs(PATH_OUT, exist_ok=True)

In [ ]:
# add back

In [ ]:
def add_white_background(image):
    img = image.convert("RGBA")
    white_background = Image.new("RGBA", img.size, (255, 255, 255, 255))
    # 將圖像黏貼到白色背景上，保持透明部分
    white_background.paste(img, (0, 0), img)
    # 轉換為 RGB 模式，去除透明度
    return white_background.convert("RGB")

In [ ]:
for filename in os.listdir(PATH_IN):
    if filename.endswith('.png'):
        open_file = os.path.join(PATH_IN, filename)
        image = Image.open(open_file)
        image_back = add_white_background(image)

        save_file = os.path.join(PATH_OUT, filename)
        save_file = save_file if save_file.endswith('.jpg') else save_file.replace('.JPG', '.jpg').replace('.webp', '.jpg').replace('.png', '.jpg')
        image_back.save(save_file, format='JPEG', quality=45)
        print(f"已保存文件: {save_file}")

已保存文件: final_3/Portrait/006_4000 × 6000.jpg
已保存文件: final_3/Portrait/008_4000 × 6000.jpg
已保存文件: final_3/Portrait/005_4000 × 6000.jpg
已保存文件: final_3/Portrait/011_4000 × 6000.jpg
已保存文件: final_3/Portrait/004_4000 × 6000.jpg
已保存文件: final_3/Portrait/013_4000 × 6000.jpg
已保存文件: final_3/Portrait/014_4000 × 6000.jpg
已保存文件: final_3/Portrait/009_4000 × 6000.jpg
已保存文件: final_3/Portrait/010_4000 × 6000.jpg
已保存文件: final_3/Portrait/020_2415 × 3680.jpg
已保存文件: final_3/Portrait/003_2624 × 3936.jpg
已保存文件: final_3/Portrait/017_4016 × 6016.jpg
已保存文件: final_3/Portrait/018_4016 × 6016.jpg
已保存文件: final_3/Portrait/019_4016 × 6016.jpg
已保存文件: final_3/Portrait/001_1433 × 1305.jpg
已保存文件: final_3/Portrait/002_2594 × 3175.jpg
已保存文件: final_3/Portrait/007_3549 × 4504.jpg
已保存文件: final_3/Portrait/012_3326 × 3028.jpg
已保存文件: final_3/Portrait/015_3295 × 3253.jpg
已保存文件: final_3/Portrait/016_2771 × 3687.jpg
已保存文件: final_3/Portrait/1.jpg
已保存文件: final_3/Portrait/2.jpg
已保存文件: final_3/Portrait/3.jpg
已保存文件: final_3/Portrait/4.jpg


In [ ]:
PATH_IN = os.path.join("milestone", "Stable Diffusion")
PATH_OUT = os.path.join("final_5", "Stable Diffusion")

# 創建輸出文件夾（如果不存在）
os.makedirs(PATH_OUT, exist_ok=True)

In [ ]:
for filename in os.listdir(PATH_IN):
    open_file = os.path.join(PATH_IN, filename)
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image = Image.open(open_file)
        if image.mode == 'RGBA':
            image = image.convert('RGB')

        # SD
        image_after = contrast_ef(image, 1.3)
        image_after = bright_ef(image_after, 1.03)
        image_after = highlights_shadows_sd(image_after)
        image_after = color_ef(image_after, 1.1)
        image_after = sharp_ef(image_after, 3)
        image_after = sharp_ef_2(image_after)

        image_after = detail_ef(image_after)

        # image_after = filter_ef_3(image_after)

        image_after = mirror_up(image)

        # image_after = gray_ef(image_after)

        save_file = os.path.join(PATH_OUT, filename)
        save_file = save_file if save_file.endswith('.jpg') else save_file.replace('.JPG', '.jpg').replace('.webp', '.jpg').replace('.png', '.jpg')
        image_after.save(save_file, format='JPEG', quality=45)
        print(f"已保存文件: {save_file}")

已保存文件: final_5/Stable Diffusion/1024x1024_ (6).jpg
已保存文件: final_5/Stable Diffusion/1024x1024_ (74).jpg
已保存文件: final_5/Stable Diffusion/1024x1024_ (19).jpg
已保存文件: final_5/Stable Diffusion/001_1360 x 2048.jpg
已保存文件: final_5/Stable Diffusion/002_1024 x 1024.jpg
已保存文件: final_5/Stable Diffusion/003_1024 x 1024.jpg
已保存文件: final_5/Stable Diffusion/004_1152 x 1728.jpg
已保存文件: final_5/Stable Diffusion/1024x1024_ (3).jpg
已保存文件: final_5/Stable Diffusion/1024x1024_ (4).jpg
已保存文件: final_5/Stable Diffusion/1024x1024_ (5).jpg
已保存文件: final_5/Stable Diffusion/1024x1024_ (7).jpg
已保存文件: final_5/Stable Diffusion/1024x1024_ (8).jpg
已保存文件: final_5/Stable Diffusion/1024x1024_ (9).jpg
已保存文件: final_5/Stable Diffusion/1024x1024_ (10).jpg
已保存文件: final_5/Stable Diffusion/1024x1024_ (11).jpg
已保存文件: final_5/Stable Diffusion/1024x1024_ (12).jpg
已保存文件: final_5/Stable Diffusion/1024x1024_ (88).jpg
已保存文件: final_5/Stable Diffusion/1024x1024_ (22).jpg
已保存文件: final_5/Stable Diffusion/1024x1024_ (84).jpg
已保存文件: final_5/Stab

In [ ]:
# 列出文件夾中的所有文件
all_files = os.listdir(PATH_OUT)

# 過濾出圖片文件 (只統計 .jpg 和 .png 的文|件)
image_files = [f for f in all_files if f.endswith(('.jpg', '.png'))]

# 打印總圖片數
print(f"文件夹中共有 {len(image_files)} 张图片")

文件夹中共有 200 张图片
